In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import math
import numpy as np
from functools import reduce
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from time import time

In [3]:
# Thumb rule: Always set the seed
np.random.seed(1234)

In [4]:
def quadratic_kappa(actual_ratings, predicted_ratings, nb_ratings=None):
    """
    This function can be used to find the value of Weighted Quadratic Kappa between two raters
    
    Arguments:
        actual_ratings: Vector of true ratings represented as integers
        predicted_ratings: Vectors of predicted ratings represented as integers
        nb_ratings: Number of unique ratings in your dataset
    
    Returns:
        weighted_kappa_score(float)
    """
    
    assert len(actual_ratings) == len(predicted_ratings), "Both the vectors should be of same length"
    
    # 1. Get the confusion mtarix
    conf_mtx = confusion_matrix(actual_ratings, predicted_ratings).astype(np.float64)
    if nb_ratings:
        nb_ratings = nb_ratings
    else:
        nb_ratings = conf_mtx.shape[0]
    
    # 2. Create a weight matrix
    weight_mtx = np.zeros((nb_ratings, nb_ratings), dtype=np.float64)
    weight_mtx +=np.arange(nb_ratings)
    weight_mtx = (weight_mtx - weight_mtx.T)**2 / ((nb_ratings-1)**2)
  
    # 3.Get the histograms for both the raters
    actual_ratings_hist = np.sum(conf_mtx, axis=1)
    predicted_ratings_hist = np.sum(conf_mtx, axis=0)
    
    
    # 4. Perform an outer product of the histograms
    out_prod = np.outer(predicted_ratings_hist, actual_ratings_hist).astype(np.float64)
    
    # 5. Normalize both- the confusion matrix and the outer product matrix
    conf_mtx /= conf_mtx.sum()
    out_prod /= out_prod.sum()
    
    # 6. Calcuate the numerator and denominator
    numerator = np.sum(conf_mtx * weight_mtx)
    denominator = np.sum(out_prod * weight_mtx)          
    
    # 7. Calculate and return weighted kappa
    return 1-(numerator/denominator)

In [5]:
# Sample example to validate if the implementation is correct or not
actuals = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 1])
preds   = np.array([0, 2, 1, 0, 0, 0, 1, 1, 2, 1])

In [9]:
# weighted kappa as per sklearn implementation
sk_imp_score = cohen_kappa_score(actuals, preds, weights='quadratic')

# weighted kappa as per our implementation
custom_imp_score = quadratic_kappa(actuals, preds)

print(np.allclose(sk_imp_score, custom_imp_score))
sk_imp_score, custom_imp_score

True


(-0.13924050632911378, -0.139240506329114)

In [10]:
# One more example
actuals = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 0])
preds = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 0])
%time print(quadratic_kappa(actuals, preds))
%time print(cohen_kappa_score(actuals, preds, weights='quadratic'))

1.0
CPU times: user 858 µs, sys: 737 µs, total: 1.6 ms
Wall time: 916 µs
1.0
CPU times: user 519 µs, sys: 162 µs, total: 681 µs
Wall time: 568 µs
